# OpenAI APIの前準備

In [ ]:
# パッケージのインストール
!pip install openai

In [ ]:
# 環境変数の準備 (左端の鍵アイコンでOPENAI_API_KEYを設定)
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI

# クライアントの準備
client = OpenAI()

In [ ]:
import json

# JSON出力ヘルパーの準備
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# Function Calling

In [ ]:
# メッセージリストの準備
messages = [
    {
        "role": "user",
        "content": "群馬県の気温は？",
    }
]

In [ ]:
# 関数定義の準備
tool = {
    "type": "function",
    "function": {
        "name": "get_current_temperature",
        "description": "特定の場所の気温を取得",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "場所 (例:東京)"},
                "unit": {"type": "string", "enum": ["c", "f"]}
            },
            "required": ["location"]
        }
    }
}

In [ ]:
# テキスト生成
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.2,
    max_tokens=500,
    tools=[tool]
)

# レスポンスメッセージの確認
response_message = response.choices[0].message
show_json(response_message)

In [ ]:
# 特定の場所の気温を取得する関数 (ダミー)
def get_current_temperature(location, unit):
    return "22度"

# 関数名と関数の関連付け
available_functions = {
    "get_current_temperature": get_current_temperature,
}

In [ ]:
import json

# レスポンスにtool_callが存在
if response_message.tool_calls:
    # レスポンスメッセージの追加
    messages.append(response_message)

    for tool_call in response_message.tool_calls:
        # 関数を呼び出す
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # 関数呼び出し結果メッセージの追加
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )

    # 再度 chat.completions.create() を呼び出す
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )

    # セカンドレスポンスメッセージの確認
    second_response_message = second_response.choices[0].message
    show_json(second_response_message)

In [ ]:
# メッセージリストの確認
print(messages)

# Parallel Function Calling

In [ ]:
# メッセージリストの準備
messages = [
    {
        "role": "user",
        "content": "京都と大阪の気温は？",
    }
]

In [ ]:
# 関数定義の準備
tool = {
    "type": "function",
    "function": {
        "name": "get_current_temperature",
        "description": "特定の場所の気温を取得",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "場所 (例:東京)"},
                "unit": {"type": "string", "enum": ["c", "f"]}
            },
            "required": ["location"]
        }
    }
}

In [ ]:
# テキスト生成
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    temperature=0.2,
    max_tokens=500,
    tools=[tool]
)

# レスポンスメッセージの確認
response_message = response.choices[0].message
show_json(response_message)

In [ ]:
# 特定の場所の気温を取得する関数 (ダミー)
def get_current_temperature(location, unit):
    return "22度"

# 関数名と関数の関連付け
available_functions = {
    "get_current_temperature": get_current_temperature,
}

In [ ]:
import json

# レスポンスにtool_callが存在
if response_message.tool_calls:
    # レスポンスメッセージの追加
    messages.append(response_message)

    for tool_call in response_message.tool_calls:
        # 関数を呼び出す
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # 関数呼び出し結果メッセージの追加
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )

    # 再度 chat.completions.create() を呼び出す
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )

    # セカンドレスポンスメッセージの確認
    second_response_message = second_response.choices[0].message
    show_json(second_response_message)